In [81]:
# import all packages

import numpy as np

In [82]:
# import data

x = np.array([
        [1, 2, 3, 4, 5],
        [5, 6, 7, 8, 10],
        [5, 6, 7, 8, 10],
        [5, 6, 7, 8, 10],
        [5, 6, 7, 8, 10],
        [5, 6, 7, 8, 10],
        [5, 6, 7, 8, 10],
        [5, 6, 7, 8, 10],
        [5, 6, 7, 8, 10],
        [5, 6, 7, 8, 10],
        ])

D2 = np.array([
        [1, 2, 3, 40, 5],
        [5, .006, .7, 800, 10],
        [5, 6, 7, 8, 10],
        [5, .6, .7, 8, 10],
        [0.05, .6, 70, 8, 10],
        [50, 6, 7, 8, 10],
        [5, 6, 7, 8, 10],
        [5, .6, 7, 8, 10],
        [500, 6, 7, 8, 10],
        [5, 6, .7, 8, 10],
        ])

In [83]:
data_shape = (x.shape[1], x.shape[0])
percent_Trunc = 0.03
r = 3
# frame_rate = 30

In [84]:
# Compression matrix generation

C_gaussian = np.random.normal(0, 1, size=data_shape)

In [85]:
# compress data

y = np.dot(C_gaussian, x)

In [86]:
# # DMD

# [U, S, V] = np.linalg.svd(y)#, full_matrices=False, compute_uv=True, hermitian=False)
# # U = U[:, 0:TruncVal]
# # S = S[0:TruncVal+1]
# # V = V[:,0:TruncVal]
# # S = np.array(S)
# # S = S * np.identity(len(S))
# Atilde = np.dot(np.dot(U.T, D2.T).T, (V.T)) / S
# Atilde = np.dot(Atilde.T, Atilde)
# [eivVec,eivVal] = np.linalg.eig(Atilde)
# eivarray = np.diag(eivVal)
# # S = np.array(S)
# # S = S * np.identity(len(S))
# X1 = np.dot(np.dot(D2, V / (S)), eivVec) 
# # X2 = np.reshape(mb.repmat(np.transpose(eivarray),np.size(x,1),1), (10, 1))  # unscaled modes
# # X3 = X1/X2
# # d = X3/x[:,0]  # scales
# # amplitude = np.absolute(d)
# # X=X1*np.diag(d)    #scaled mode

# # end of DMD

In [87]:
# DMD

[U, S, V] = np.linalg.svd(y, full_matrices=False, compute_uv=True, hermitian=False)
# U = U[:, 0:TruncVal]
# S = S[0:TruncVal+1]
# V = V[:,0:TruncVal]
# S = np.array(S)
# S = S * np.identity(len(S))
Atilde = U[:, : r].conj().T @ np.dot(D2.T, D2) @ V[: r, :].conj().T * np.reciprocal(S[: r])
# Atilde = np.dot(np.dot(U.T, D2.T).T, (V.T)) / S
Atilde = np.dot(Atilde.T, Atilde)
[eivVec,eivVal] = np.linalg.eig(Atilde)
eivarray = np.diag(eivVal)
# S = np.array(S)
# S = S * np.identity(len(S))
# X1 = np.dot(D2 @ V[: r, :].conj().T * np.reciprocal(S[: r]), eivVec) 
# X2 = np.reshape(mb.repmat(np.transpose(eivarray),np.size(x,1),1), (10, 1))  # unscaled modes
# X3 = X1/X2
# d = X3/x[:,0]  # scales
# amplitude = np.absolute(d)
# X=X1*np.diag(d)    #scaled mode

sp_modes = D2 @ V[: r, :].conj().T @ np.diag(np.reciprocal(S[: r])) @ eivVec

# end of DMD

In [88]:
modeNorm = np.zeros(r)
for i in (0, r-1):
    modeNorm[i] = np.linalg.norm(X[i]) # norm of each mode

mag = np.absolute(eivarray)
weigh = mag[1:-1]/np.sum(mag)
delta_t = 1/frame_rate   
omega = np.log(eivarray)/delta_t
growth = np.real(omega)
frequency = np.imag(omega)/(2*np.pi)

/tmp/ipykernel_346/2542409379.py:8: RuntimeWarning: invalid value encountered in log
  omega = np.log(eivarray)/delta_t


In [89]:
# #

# freq1 = np.where(frequency>=0)     # eliminating negative frequencies
# amplitude1 = amplitude[freq1]  # eliminating amplitudes corresponding to -ve frequencies
# d1 = d[freq1]    # similar operations below for different parameters
# omega1 = omega[freq1]
# growth1 = growth[freq1]
# # weigh1 = weigh[freq1]
# # modeNorm1 = modeNorm[:,freq1]
# frequency1 = frequency[freq1]
# Xplus = X[:freq1]